In [1]:
%matplotlib inline
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
np.random.seed(123)
tf.random.set_seed(123)

mnist = datasets.mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = (x_train.reshape(-1, 28, 28) / 255).astype(np.float32)
x_test = (x_test.reshape(-1, 28, 28) / 255).astype(np.float32)

x_train, x_val, t_train, t_val = \
    train_test_split(x_train, t_train, test_size=0.2)
print("x training array shape : ", x_train.shape)
print(x_train)
print("t train array shape : ", t_train.shape)
print(t_train)

x training array shape :  (48000, 28, 28)
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 

In [2]:
np.random.seed(123)
tf.random.set_seed(123)

file_paths = glob.glob(".\\data\\*")
print(file_paths)

['.\\data\\20200319_not_seating.csv', '.\\data\\20200319_seating.csv', '.\\data\\20200322a_not_seating.csv', '.\\data\\20200323a_seating.csv', '.\\data\\20200323b_not_seating.csv']


In [4]:
category =np.empty((0,1), float)
rssi =np.empty((0,100), float)

for file in file_paths:
    d = np.loadtxt(file, delimiter=',')
    category_tmp, rssi_tmp = np.hsplit(d, [1])
    rssi = np.concatenate([rssi, rssi_tmp], axis=0)
    category = np.concatenate([category, category_tmp], axis=0)

rssi = rssi * (-1) / 128
print("rssi array shape : ", rssi.shape)
#print(rssi)
category = tf.keras.utils.to_categorical(category, 2)
print("category array shape : ", category.shape)
#print(category)

x_train, x_val, t_train, t_val = train_test_split(rssi, category, test_size=0.2)
print("x training array shape : ", x_train.shape)
print(x_train)
print("t training array shape : ", t_train.shape)
print(t_train)
print("rssi test array shape : ", x_val.shape)
print("category test array shape : ", t_val.shape)

rssi array shape :  (1507, 100)
category array shape :  (1507, 2)
x training array shape :  (1205, 100)
[[0.4375    0.3359375 0.3359375 ... 0.40625   0.40625   0.390625 ]
 [0.34375   0.359375  0.359375  ... 0.3359375 0.3671875 0.3828125]
 [0.390625  0.3203125 0.328125  ... 0.3359375 0.3359375 0.34375  ]
 ...
 [0.421875  0.3359375 0.375     ... 0.3828125 0.453125  0.375    ]
 [0.375     0.3828125 0.3984375 ... 0.3671875 0.390625  0.3828125]
 [0.4296875 0.4375    0.4375    ... 0.40625   0.453125  0.421875 ]]
t training array shape :  (1205, 2)
[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
rssi test array shape :  (302, 100)
category test array shape :  (302, 2)


In [5]:
class BiRNN(Model):
    def __init__(self, hidden_dim):
        super().__init__()
        self.l1 = Bidirectional(LSTM(hidden_dim, activation='tanh',
                                     recurrent_activation='sigmoid',
                                     kernel_initializer='glorot_normal',
                                     recurrent_initializer='orthogonal'),
                                merge_mode='concat')
        #self.l2 = Dense(10, kernel_initializer='glorot_normal',
        #                activation='softmax')
        self.l2 = Dense(2, kernel_initializer='glorot_normal',
                        activation='softmax')

    def call(self, x):
        h = self.l1(x)
        y = self.l2(h)
        return y

model = BiRNN(25)

criterion = losses.SparseCategoricalCrossentropy()
optimizer = optimizers.Adam(learning_rate=0.001,
                                beta_1=0.9, beta_2=0.999, amsgrad=True)
train_loss = metrics.Mean()
train_acc = metrics.SparseCategoricalAccuracy()
val_loss = metrics.Mean()
val_acc = metrics.SparseCategoricalAccuracy()

In [6]:
def compute_loss(t, y):
    return criterion(t, y)

def train_step(x, t):
    with tf.GradientTape() as tape:
        preds = model(x)
        loss = compute_loss(t, preds)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss)
    train_acc(t, preds)

    return loss

def val_step(x, t):
    preds = model(x)
    loss = compute_loss(t, preds)
    val_loss(loss)
    val_acc(t, preds)
    
#epochs = 1000
epochs = 10
batch_size = 1
n_batches_train = x_train.shape[0] // batch_size
n_batches_val = x_val.shape[0] // batch_size
hist = {'loss': [], 'accuracy': [],
        'val_loss': [], 'val_accuracy': []}
es = EarlyStopping(patience=5, verbose=1)

In [9]:
for epoch in range(epochs):
    x_, t_ = shuffle(x_train, t_train)

    for batch in range(n_batches_train):
        start = batch * batch_size
        end = start + batch_size
        train_step(x_[start:end], t_[start:end])

    for batch in range(n_batches_val):
        start = batch * batch_size
        end = start + batch_size
        val_step(x_val[start:end], t_val[start:end])

    hist['loss'].append(train_loss.result())
    hist['accuracy'].append(train_acc.result())
    hist['val_loss'].append(val_loss.result())
    hist['val_accuracy'].append(val_acc.result())

    print('epoch: {}, loss: {:.3}, acc: {:.3f}'
          ', val_loss: {:.3}, val_acc: {:.3f}'.format(
              epoch+1,
              train_loss.result(),
              train_acc.result(),
              val_loss.result(),
              val_acc.result()
          ))

#    if es(val_loss.result()):
#            break


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: Input 0 of layer bidirectional is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [1, 100]

test_loss: 0.080, test_acc: 0.976


In [8]:
#正答率
plt.plot(hist['accuracy'])
plt.plot(hist['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#loss
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# テストデータの評価
test_loss = metrics.Mean()
test_acc = metrics.SparseCategoricalAccuracy()

def test_step(x, t):
    preds = model(x)
    loss = compute_loss(t, preds)
    test_loss(loss)
    test_acc(t, preds)

    return loss

test_step(x_test, t_test)

print('test_loss: {:.3f}, test_acc: {:.3f}'.format(
    test_loss.result(),
    test_acc.result()
))